In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import model_selection, preprocessing
import xgboost as xgb
from datetime import timedelta
import datetime
import math
from sklearn import model_selection
# from sklearn import proprocessing
from sklearn import linear_model
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV

In [2]:
cd /Users/williamzhou/Downloads/RussianHousing-master

/Users/williamzhou/Downloads/RussianHousing-master


In [3]:
train_missing=pd.read_csv('./data/processed/Clean0517/train_xgb.csv')
test_missing= pd.read_csv('./data/processed/Clean0517/test_xgb.csv')


In [5]:
train_missing['price_full_sq'] = (train_missing['price_doc']/train_missing['full_sq'].astype(float)).astype(int)
y_train_missing = train_missing['price_full_sq']
test_id = test_missing.id

In [6]:
x_train_missing = train_missing.drop(['price_full_sq','price_doc','id','timestamp'],axis=1)
x_test_missing = test_missing.drop(['id','timestamp'],axis=1)

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}



x_train_missing_subset = x_train_missing
x_test_missing_subset = x_test_missing
dtrain_subset = xgb.DMatrix(x_train_missing_subset, y_train_missing)
dtest_subset =  xgb.DMatrix(x_test_missing_subset)
cv_output = xgb.cv(xgb_params, dtrain_subset, 
                       num_boost_round=1000, 
                       early_stopping_rounds=20,
                       verbose_eval=50, show_stdv=False)
test_rmse = cv_output.loc[len(cv_output)-1,'test-rmse-mean']
print(test_rmse)


[0]	train-rmse:137644	test-rmse:137656
[50]	train-rmse:42357.7	test-rmse:44128.3
[100]	train-rmse:38930.5	test-rmse:41692.2
[150]	train-rmse:37374.2	test-rmse:41133.8
[200]	train-rmse:36159.7	test-rmse:40803.7
[250]	train-rmse:35089.7	test-rmse:40634.4
[300]	train-rmse:34193.7	test-rmse:40534.9
[350]	train-rmse:33339	test-rmse:40468.8
[400]	train-rmse:32548.1	test-rmse:40432.6
[450]	train-rmse:31777.1	test-rmse:40406.5
40402.529948


In [ ]:
num_boost_rounds= len(cv_output)
print(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain_subset, num_boost_round= num_boost_rounds)
print('Finish XGBoost training')

     test-rmse-mean  test-rmse-std  train-rmse-mean  train-rmse-std
0     137656.375000     518.822874    137644.328125      251.331099
1     131583.458333     532.695565    131547.802083      244.129053
2     125837.625000     562.104868    125783.411458      237.543497
3     120397.791667     583.676207    120323.851563      234.887814
4     115260.992187     612.720321    115177.606771      229.834781
5     110431.585937     618.932120    110323.528646      240.601518
6     105876.822916     627.069519    105749.372396      248.774887
7     101590.872396     630.091708    101442.565105      267.213273
8      97536.755209     658.606974     97367.846354      258.870396
9      93732.565104     687.720695     93531.617188      271.276129
10     90145.796875     692.632116     89917.565104      283.823557
11     86782.260417     677.268534     86527.562500      308.958228
12     83628.239584     690.656720     83342.747396      317.553347
13     80668.294271     687.497602     80345.208

In [ ]:
y_predict = model.predict(dtest_subset)
y_predic_all_sq = (y_predict)*x_test_missing['full_sq']
df_sub = pd.DataFrame({'id': test_id, 'price_doc': y_predic_all_sq})
df_sub.to_csv('./sub.csv', index=False)
df_sub.head()